Лабораторна робота №4

Роботу виконали студенти групи ФБ - 35: Церман Марія, Кохта Андрій, Ворона Сергій.

In [1]:
import random

Генерація простих чисел та розбиття на пари

In [2]:
def power(a, b, m):
    res = 1
    a %= m
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % m
        a = (a * a) % m
        b //= 2

    return res

def miller_rabin_test(n, k=40):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False

    s = 0
    d = n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = power(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for r in range(s - 1):
            x = power(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True


def get_random_prime(bits):
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << (bits - 1)) | 1

        small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]
        is_composite = False
        for p in small_primes:
            if candidate % p == 0 and candidate != p:
                is_composite = True
                break

        if is_composite:
            continue

        if miller_rabin_test(candidate):
            return candidate

In [3]:
p, q, p1, q1 = get_random_prime(256), get_random_prime(256), get_random_prime(256), get_random_prime(256)

In [4]:
def check_keys(p, q, p1, q1):
    n = p * q
    n1 = p1 * q1

    if n > n1:
        return (p1, q1), (p, q), n1, n
    else:
        return (p, q), (p1, q1), n, n1

In [5]:
pair_A, pair_B, mult_A, mult_B = check_keys(p, q, p1, q1)
print("Пара для абонента А:")
print(f"p: {pair_A[0]}\n"
      f"q: {pair_A[1]}\n"
      f"pq: {mult_A}\n")

print("Пара для абонента B:")
print(f"p1: {pair_B[0]}\n"
      f"q1: {pair_B[1]}\n"
      f"pq1: {mult_B}\n")

print("pq < pq1:", ["Ні", "Так"][mult_A <= mult_B])

Пара для абонента А:
p: 70303008252513781100090105658282255378807316292921360212807120618075856751041
q: 107885278225073179461317321405886951707136533170091933549782219120698480976279
pq: 7584659605382065065902873781326174119093580148287430919175413296101240120706550705043207349697308733831599240716335969030328106143990884421866470729556439

Пара для абонента B:
p1: 89973177781262507752640534883629741930678276182860030607710947283195823291609
q1: 98182475052434044438598483701579936011724552480550204375859780225541982244359
pq1: 8833789282897019237925575555925318004270593629772502574924256059556840443174479423328939048142093173605755401434041109963522030158175232505706012752283631

pq < pq1: Так


Генерація пар ключів(публічного та секретного) для RSA

In [6]:
def euclid_extended(a, n):
    r0, r1 = a, n
    u0, v0 = 1, 0
    u1, v1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        (r0, r1) = (r1, r0 - q * r1)

        (u0, u1) = (u1, u0 - q * u1)
        (v0, v1) = (v1, v0 - q * v1)

    gcd = r0
    u = u0
    v = v0

    return gcd, u, v

def generate_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    gcd, u, _ = euclid_extended(e, phi)

    while gcd != 1:
        e = random.randrange(3, phi, 2)
        gcd, u, _ = euclid_extended(e, phi)

    d = u % phi

    return (e, n), (d, p, q)

In [7]:
public_A, private_A = generate_key_pair(pair_A[0], pair_A[1])
public_B, private_B = generate_key_pair(pair_B[0], pair_B[1])

for pub, priv, name in [(public_A, private_A, "A"), (public_B, private_B, "B")]:
    e, n = pub
    d, p, q = priv

    print(f"Абонент {name}:")
    print("Відкритий ключ")
    print(f"e: {e}\n"
          f"n: {n}\n")

    print("Приватний ключ")
    print(f"d: {d}\n"
          f"p: {p}\n"
          f"q: {q}\n")

Абонент A:
Відкритий ключ
e: 65537
n: 7584659605382065065902873781326174119093580148287430919175413296101240120706550705043207349697308733831599240716335969030328106143990884421866470729556439

Приватний ключ
d: 4607480785044643093271361383373630834938334267110474995105231459242604508683066094528570506951422624758798087598246212101570121863272485524355216881326593
p: 70303008252513781100090105658282255378807316292921360212807120618075856751041
q: 107885278225073179461317321405886951707136533170091933549782219120698480976279

Абонент B:
Відкритий ключ
e: 65537
n: 8833789282897019237925575555925318004270593629772502574924256059556840443174479423328939048142093173605755401434041109963522030158175232505706012752283631

Приватний ключ
d: 3071075194493578990751732814535337983265959767165672805698677847032104805793415204277467450915121621032914012679417015477678544888078472038000621212241921
p: 89973177781262507752640534883629741930678276182860030607710947283195823291609
q: 981824750524340444

Шифрування, розшифрування, цифровий підпис, перевірка

In [8]:
def Encrypt(message, public_key):
    e, n = public_key
    if message >= n:
        raise ValueError("Помилка: Повідомлення M має бути меншим за модуль n")

    return power(message, e, n)

def Decrypt(ciphertext, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(ciphertext, d, n)

def Sign(message, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(message, d, n)

def Verify(message, signature, public_key):
    e, n = public_key
    check_val = power(signature, e, n)

    return check_val == message

In [9]:
subscribers = [("А", public_A, private_A), ("В", public_B, private_B)]

for name, pub_key, priv_key in subscribers:
    # Генерація повідомлення
    n_module = pub_key[1]
    message = random.randint(100, n_module // 2)
    print(f"Відкрите повідомлення {name}: {message}")

    # Шифрування
    crypto = Encrypt(message, pub_key)
    print(f"Криптограма {name}: {crypto}")

    # Розшифрування
    decrypted = Decrypt(crypto, priv_key)
    print(f"Розшифроване повідомлення {name}: {decrypted}")
    print(["Не співпадають\n", "Співпадають\n"][decrypted == message])

    # Цифровий підпис
    signature = Sign(message, priv_key)
    print(f"Цифровий підпис повідомлення {name}: {signature}")

    # Перевірка підпису
    is_valid = Verify(message, signature, pub_key)
    status = (["!Невалідний!", "!Валідний!"][is_valid])
    print(f"Підпис {name}: {status}\n")

Відкрите повідомлення А: 1957698381593193188153439528029161849298063992155639678463314351818551374724007214153917188827356796072282249203591541258376386349413990105230198495696054
Криптограма А: 4403837726745493218739084164974334164650868369878224207632362252965735362885453388179416155645687920906033843666456551412926323578567733761303606281633585
Розшифроване повідомлення А: 1957698381593193188153439528029161849298063992155639678463314351818551374724007214153917188827356796072282249203591541258376386349413990105230198495696054
Співпадають

Цифровий підпис повідомлення А: 7047703015236338977403640163466183256280711243681337858796974754075412203393417321033931259536768854248956312640608746224595323457947015546514080652709390
Підпис А: !Валідний!

Відкрите повідомлення В: 2367040957807357834837765340802215317706066561216460722422170622660606802902085526842924974830659265090807391212477352244378676877060921017555880270582569
Криптограма В: 2534356713387929620991754035075262993745027484017

Конфіденційне розсилання ключів

In [10]:
def SendKey(k, private_key_A, public_key_B):
    S = Sign(k, private_key_A)
    k1 = Encrypt(k, public_key_B)
    S1 = Encrypt(S, public_key_B)

    return k1, S1

def ReceiveKey(package, private_key_B, public_key_A):
    k1, S1 = package
    k = Decrypt(k1, private_key_B)
    S = Decrypt(S1, private_key_B)
    is_valid = Verify(k, S, public_key_A)

    if is_valid:
        return k, "Підпис вірний, ключ отримано!"
    else:
        return None, "Підпис невірний!"

In [11]:
# 0<k<n
n_a = public_A[1]
secret_k = random.randint(100, n_a // 2)
print(f"Секретний ключ k: {secret_k}\n")

print("Відправляє А:")
package = SendKey(secret_k, private_A, public_B)
print(f"Шифрований ключ: {package[0]}\n"
      f"Цифровий підпис: {package[1]}\n")

print("Отримує B:")
received_k, status_msg = ReceiveKey(package, private_B, public_A)
print(f"Відкритий ключ k: {received_k}")
print(status_msg)

Секретний ключ k: 1166021370358548910534139437832066676205599666972718386930672141764087995839678136982221827392933074009773661105691080649391187667003361219811711852658008

Відправляє А:
Шифрований ключ: 2011732000277930813712303628419389389825320163119108455577868423749997247877138821645034320125523299831517960107127413262180389873341618925522717387557638
Цифровий підпис: 972358541082970251280608680850741744489868206948472729465770411658659093399638227845465385585556734658382610095423756127780478386686666262413570659016974

Отримує B:
Відкритий ключ k: 1166021370358548910534139437832066676205599666972718386930672141764087995839678136982221827392933074009773661105691080649391187667003361219811711852658008
Підпис вірний, ключ отримано!
